In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import DenseNet121, DenseNet169

import gc
import cv2
import numpy as np 
import pandas as pd 
import random 
import os
from tqdm.auto import tqdm
import pickle
import time
import albumentations
from albumentations.core.transforms_interface import DualTransform, ImageOnlyTransform
from albumentations.augmentations import functional as F

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


# Load Data & Aug

In [0]:
'''96x96'''
'''Load image data and labels'''

tot_file_0 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/train_0_96x96", 'rb')
tot_data_0 = pickle.load(tot_file_0)
tot_file_1 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/train_1_96x96", 'rb')
tot_data_1 = pickle.load(tot_file_1)
tot_file_2 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/train_2_96x96", 'rb')
tot_data_2 = pickle.load(tot_file_2)
tot_file_3 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/train_3_96x96", 'rb')
tot_data_3 = pickle.load(tot_file_3)
tot_data = pd.concat([pd.DataFrame(tot_data_0), pd.DataFrame(tot_data_1), pd.DataFrame(tot_data_2), pd.DataFrame(tot_data_3)],ignore_index=True)
# print(tot_data.shape)
T = tot_data.shape[0]

del tot_file_0
del tot_file_1
del tot_file_2
del tot_file_3

tot_csv = pd.read_csv("/content/drive/My Drive/Untitled folder/bengaliai-cv19/train.csv")
tot_labels = [tot_csv.loc[:T, ['grapheme_root']], 
              tot_csv.loc[:T, ['vowel_diacritic']], 
              tot_csv.loc[:T, ['consonant_diacritic']]]

In [0]:
'''Load image data and labels'''

tot_file_0 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/Resized/train_0_resized", 'rb')
tot_data_0 = pickle.load(tot_file_0)
tot_file_1 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/Resized/train_1_resized", 'rb')
tot_data_1 = pickle.load(tot_file_1)
tot_file_2 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/Resized/train_2_resized", 'rb')
tot_data_2 = pickle.load(tot_file_2)
tot_file_3 = open("/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/Resized/train_3_resized", 'rb')
tot_data_3 = pickle.load(tot_file_3)
tot_data = pd.concat([tot_data_0, tot_data_1, tot_data_2, tot_data_3],ignore_index=True)
# print(tot_data.shape)
T = tot_data.shape[0]

del tot_file_0
del tot_file_1
del tot_file_2
del tot_file_3

tot_csv = pd.read_csv("/content/drive/My Drive/Untitled folder/bengaliai-cv19/train.csv")
tot_labels = [tot_csv.loc[:T, ['grapheme_root']], 
              tot_csv.loc[:T, ['vowel_diacritic']], 
              tot_csv.loc[:T, ['consonant_diacritic']]]

In [0]:
# Loading the data, run once every load

"""
7 consonant
11 vowel
168 root
"""

SIZE=96

def returndata(train_size, T, tot_data, tot_labels, tot_csv, SIZE):
    train = tot_data.loc[:train_size-1, ]
    
    train = np.reshape(train.values, (train.shape[0], SIZE, SIZE, 1))

    root_train_labels, vowel_train_labels, consonant_train_labels = (tot_labels[0][:train_size], 
                                       tot_labels[1][:train_size], 
                                       tot_labels[2][:train_size])
    tot_csv = tot_csv[:train_size]

    train, test, root_train_labels, root_test_labels, vowel_train_labels, vowel_test_labels, consonant_train_labels, consonant_test_labels, train_csv, test_csv = train_test_split(train, root_train_labels, vowel_train_labels, consonant_train_labels, tot_csv, test_size=1/6, random_state=666)
    test, valid, root_test_labels, root_valid_labels, vowel_test_labels, vowel_valid_labels, consonant_test_labels, consonant_valid_labels, test_csv, valid_csv = train_test_split(test, root_test_labels, vowel_test_labels, consonant_test_labels, test_csv, test_size=0.5, random_state=666)

    return ((train, valid, test), 
            (root_train_labels, root_valid_labels, root_test_labels), 
            (vowel_train_labels, vowel_valid_labels, vowel_test_labels), 
            (consonant_train_labels, consonant_valid_labels, consonant_test_labels),
            (tot_data, tot_labels), (train_csv, valid_csv, test_csv))

TOT_DATA = returndata(T, T, tot_data, tot_labels, tot_csv, SIZE)

((train_data, valid_data, test_data), 
(root_train_labels, root_valid_labels, root_test_labels), 
(vowel_train_labels, vowel_valid_labels, vowel_test_labels), 
(consonant_train_labels, consonant_valid_labels, consonant_test_labels),
(tot_data, tot_labels), (train_csv, valid_csv, test_csv)) = TOT_DATA

In [0]:
class MulticlassGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self, X, Y,
             batch_size = 64,
             shuffle = True):
        
        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in Y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(X, targets, 
                                         batch_size = batch_size,
                                         shuffle = shuffle):
            target_dict = {}
            for i, output in enumerate(ordered_outputs):
                target_dict[output] = flowy[:, i]

            yield flowx, target_dict


## Aug

In [0]:
class GridMask(DualTransform):
    """GridMask augmentation for image classification and object detection.

    Args:
        num_grid (int): number of grid in a row or column.
        fill_value (int, float, lisf of int, list of float): value for dropped pixels.
        rotate ((int, int) or int): range from which a random angle is picked. If rotate is a single int
            an angle is picked from (-rotate, rotate). Default: (-90, 90)
        mode (int):
            0 - cropout a quarter of the square of each grid (left top)
            1 - reserve a quarter of the square of each grid (left top)
            2 - cropout 2 quarter of the square of each grid (left top & right bottom)

    Targets:
        image, mask

    Image types:
        uint8, float32

    Reference:
    |  https://arxiv.org/abs/2001.04086
    |  https://github.com/akuxcw/GridMask
    """

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = F.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        return image

    def get_params_dependent_on_targets(self, params):
        img = params['image']
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

# Model

In [0]:
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout

SIZE = 96

keras.backend.clear_session()

densenet = DenseNet121(weights='imagenet', include_top=False)

inputs = keras.Input(shape=(SIZE, SIZE, 1))
model = Conv2D(3, (3, 3), padding="same")(inputs)

model = densenet(model)

model = GlobalAveragePooling2D()(model)
model = BatchNormalization()(model)
model = Dropout(0.3)(model)
dense = Dense(1024, activation="relu")(model)
# dense = Dropout(0.3)(dense)
# dense = Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(dense)

root = keras.layers.Dense(168, name='root', activation="softmax")(dense)
vowel = keras.layers.Dense(11, name='vowel', activation="softmax")(dense)
consonant = keras.layers.Dense(7, name='consonant', activation="softmax")(dense)

model = keras.models.Model(inputs=inputs, outputs=[root, vowel, consonant])

# sgd = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
class_weight = {'root':4, 'vowel':3, 'consonant':3}

model.compile(optimizer="adam", loss={'root': 'sparse_categorical_crossentropy',
                    'vowel': 'sparse_categorical_crossentropy',
                    'consonant': 'sparse_categorical_crossentropy'},
              metrics={'root': 'accuracy',
                       'vowel': 'accuracy',
                       'consonant': 'accuracy'},
              loss_weights=class_weight)

In [0]:
transforms_train = albumentations.Compose([
    GridMask(num_grid=3, rotate=8, p=1),
])

def transforms(image):
  image = cv2.resize(image, (SIZE, SIZE))
  temp = transforms_train(image=image)
  image = temp['image']
  return image[:,:,np.newaxis]

In [0]:
batch_size = 64

lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor = 'val_root_loss',
                             patience = 2,
                             factor = 0.5,
                             min_lr = 1e-5,
                             verbose = 1)

earlystop = keras.callbacks.EarlyStopping(monitor="val_root_loss", patience=6, restore_best_weights=True)

checkpoint = keras.callbacks.ModelCheckpoint("/content/drive/My Drive/Colab Notebooks/bengaliai-cv19/saved_models/densenet_3.h5", 
                                 monitor = 'val_root_loss', 
                                 verbose = 0, save_best_only=True, 
                                 mode = 'min',
                                 save_weights_only = True)

In [0]:
del tot_data_0
del tot_data_1
del tot_data_2
del tot_data_3

In [0]:
train_datagen = MulticlassGenerator(
    rescale=1./255, 
    rotation_range=8,
    shear_range=2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    preprocessing_function=transforms,
)

datagen = MulticlassGenerator(rescale=1./255)

train_datagen.fit(train_data, augment=True)

train_gen = train_datagen.flow(train_data, {'root':root_train_labels, 'vowel':vowel_train_labels, 'consonant':consonant_train_labels}, 
                         batch_size=batch_size, shuffle=True)
valid_gen = datagen.flow(valid_data, {'root':root_valid_labels,'vowel':vowel_valid_labels,'consonant':consonant_valid_labels}, 
                         batch_size=batch_size, shuffle=False)

gc.collect()

263

In [0]:
# Training Loop
train_range = 4
SIZE = 128
seed = 6666

histories = []

tot_csv = pd.read_csv("/content/drive/My Drive/Untitled folder/bengaliai-cv19/train.csv")
tot_labels = [tot_csv.loc[:, ['grapheme_root']], 
              tot_csv.loc[:, ['vowel_diacritic']], 
              tot_csv.loc[:, ['consonant_diacritic']]]

for i in range(train_range):
  train_file = pickle.load(open(f"/content/drive/My Drive/Untitled folder/bengaliai-cv19/Resized/train_{i}_{SIZE}x{SIZE}", 'rb'))
  T = train_file.shape[0]
  train_data = np.reshape(train_file, (train_file.shape[0], SIZE, SIZE, 1))
  del train_file
  root_train_labels, vowel_train_labels, consonant_train_labels = (tot_labels[0][50210*i:50210*(i+1)], 
                                    tot_labels[1][50210*i:50210*(i+1)], 
                                    tot_labels[2][50210*i:50210*(i+1)])
  train_data, test_data, root_train_labels, root_test_labels, vowel_train_labels, vowel_test_labels, consonant_train_labels, consonant_test_labels = train_test_split(train_data, root_train_labels, vowel_train_labels, consonant_train_labels, test_size=0.1, random_state=seed)
  test_data, valid_data, root_test_labels, root_valid_labels, vowel_test_labels, vowel_valid_labels, consonant_test_labels, consonant_valid_labels = train_test_split(test_data, root_test_labels, vowel_test_labels, consonant_test_labels, test_size=0.5, random_state=seed)

  train_datagen = MulticlassGenerator(
      rescale=1./255, 
      rotation_range=8,
      shear_range=2,
      width_shift_range=0.15,
      height_shift_range=0.15,
      preprocessing_function=transforms
  )

  datagen = MulticlassGenerator(rescale=1./255)

  train_datagen.fit(train_data)

  train_gen = train_datagen.flow(train_data, {'root':root_train_labels, 'vowel':vowel_train_labels, 'consonant':consonant_train_labels}, 
                         batch_size=batch_size, shuffle=True)
  valid_gen = datagen.flow(valid_data, {'root':root_valid_labels,'vowel':vowel_valid_labels,'consonant':consonant_valid_labels}, 
                         batch_size=batch_size, shuffle=False)

  history = model.fit(train_gen, epochs=60, verbose=1,
          validation_data=valid_gen,
          validation_steps=valid_data.shape[0] // (batch_size),
          steps_per_epoch=train_data.shape[0] // (batch_size), 
          callbacks=[lr_scheduler, earlystop 
                     ,checkpoint
                     ]
          )
  del train_data
  del test_data
  del valid_data
  
  histories.append(history)


  gc.collect()

Epoch 1/60
706/706 [==============================] - 175s 248ms/step - loss: 13.1189 - root_loss: 2.3095 - vowel_loss: 0.7493 - consonant_loss: 0.5443 - root_acc: 0.4348 - vowel_acc: 0.7486 - consonant_acc: 0.8212 - val_loss: 6.5598 - val_root_loss: 1.2414 - val_vowel_loss: 0.2880 - val_consonant_loss: 0.2434 - val_root_acc: 0.6671 - val_vowel_acc: 0.9075 - val_consonant_acc: 0.9191
Epoch 2/60
706/706 [==============================] - 131s 186ms/step - loss: 5.7271 - root_loss: 0.9416 - vowel_loss: 0.3708 - consonant_loss: 0.2828 - root_acc: 0.7323 - vowel_acc: 0.8801 - consonant_acc: 0.9076 - val_loss: 4.3653 - val_root_loss: 0.7677 - val_vowel_loss: 0.2664 - val_consonant_loss: 0.1652 - val_root_acc: 0.7879 - val_vowel_acc: 0.9174 - val_consonant_acc: 0.9469
Epoch 3/60
706/706 [==============================] - 131s 185ms/step - loss: 4.6017 - root_loss: 0.7403 - vowel_loss: 0.3086 - consonant_loss: 0.2383 - root_acc: 0.7880 - vowel_acc: 0.9023 - consonant_acc: 0.9229 - val_loss: 3

In [0]:
# earlystop = keras.callbacks.EarlyStopping(monitor="val_root_loss", patience=3, restore_best_weights=True)
# model.load_weights("/content/drive/My Drive/Colab Notebooks/bengaliai-cv19/saved_models/densenet_3.h5")
# load = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/bengaliai-cv19/saved_models/densenet_aug_1.h5")

history_res = model.fit(train_gen, epochs=60, verbose=1,
          validation_data=valid_gen,
          validation_steps=valid_data.shape[0] // (batch_size),
          steps_per_epoch=train_data.shape[0] // (batch_size), 
          callbacks=[lr_scheduler, earlystop 
                     ,checkpoint
                     ]
          )

Epoch 1/60
1307/1307 [==============================] - 331s 254ms/step - loss: 8.4933 - root_loss: 1.4659 - vowel_loss: 0.5026 - consonant_loss: 0.3740 - root_acc: 0.6232 - vowel_acc: 0.8352 - consonant_acc: 0.8778 - val_loss: 2.9166 - val_root_loss: 0.5038 - val_vowel_loss: 0.1548 - val_consonant_loss: 0.1457 - val_root_acc: 0.8599 - val_vowel_acc: 0.9543 - val_consonant_acc: 0.9560
Epoch 2/60
1307/1307 [==============================] - 286s 219ms/step - loss: 3.8052 - root_loss: 0.6108 - vowel_loss: 0.2541 - consonant_loss: 0.1999 - root_acc: 0.8255 - vowel_acc: 0.9219 - consonant_acc: 0.9363 - val_loss: 2.2251 - val_root_loss: 0.3634 - val_vowel_loss: 0.1235 - val_consonant_loss: 0.1337 - val_root_acc: 0.8990 - val_vowel_acc: 0.9657 - val_consonant_acc: 0.9610
Epoch 3/60
1307/1307 [==============================] - 283s 217ms/step - loss: 3.2029 - root_loss: 0.5081 - vowel_loss: 0.2182 - consonant_loss: 0.1721 - root_acc: 0.8533 - vowel_acc: 0.9332 - consonant_acc: 0.9451 - val_lo

In [0]:
import os
model_name = 'densenet_aug_3.h5'
save_dir = os.path.join("/content/drive/My Drive/Colab Notebooks/bengaliai-cv19/", 'saved_models')

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/drive/My Drive/Colab Notebooks/bengaliai-cv19/saved_models/densenet_aug_3.h5 


In [0]:
load = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/bengaliai-cv19/saved_models/densenet_aug_1.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
load.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 96, 96, 3)    30          input_2[0][0]                    
__________________________________________________________________________________________________
densenet121 (Model)             multiple             7037504     conv2d[0][0]                     
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 1024)         0           densenet121[1][0]                
______________________________________________________________________________________________

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 96, 96, 3)    30          input_2[0][0]                    
__________________________________________________________________________________________________
densenet121 (Model)             multiple             7037504     conv2d[0][0]                     
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 1024)         0           densenet121[1][0]                
______________________________________________________________________________________________

In [0]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] 
row_id=[] 

model_1 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/bengaliai-cv19/saved_models/resnet34_aug_1.h5")
model_2 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/bengaliai-cv19/saved_models/densenet_aug_1.h5")

for i in range(4):
    testfile_0 = pd.read_parquet('/content/drive/My Drive/Untitled folder/bengaliai-cv19/test_image_data_{}.parquet'.format(i))
    testfile_0.set_index('image_id', inplace=True)
    
    testdata = np.reshape(testfile_0.values, (testfile_0.shape[0], 137, 236, 1))

    new_data = [0] * testdata.shape[0]

    for j in tqdm(range(testdata.shape[0])):

        curpic = testdata[j, :, :, 0]
        # curpic = np.array(curpic,dtype=np.float)
        (x1, x2, y1, y2) = findEdge(curpic, 200)
        
        curpic_2 = np.stack([i[x1:x2] for i in curpic[y1:y2]],axis=0)
        del curpic
        newpic = changeshape(curpic_2, 96, 96) / 255
        del curpic_2
        new_data[j] = newpic
        del newpic

    new_data = np.reshape(np.stack(new_data), (len(new_data), 96, 96, 1))
    
    preds_1 = model_1.predict(new_data)
    preds_2 = model_2.predict(new_data)

    for k, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax((preds_1[k]+preds_2[k])/2, axis=1)

    for l,id in enumerate(testfile_0.index.values):  
        for comp in components:
            id_sample=str(id)+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][l])
    del testfile_0

df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample